Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `???` or "YOUR ANSWER HERE".

---

# CSCI 3155: Assignment 6

Topics Covered: Function calls, closures, currying, recursion and circular scopes.

__Name__: WRITE YOUR NAME HERE

In [8]:
// TEST HELPER
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"\n*** Tests Passed (1 point) ***\n")
    else print(s"\n*** Tests Passed ($points points) ***\n")
}

defined function passed

## Problem 1 (30 Points): Mutual Recursion in Lettuce

In class, we have explored recursive functions in lettuce using the _let rec_ syntax. In this problem, we will 
explore, mutually recursive function, specifically two mutually recursive functions.

Consider: 

~~~
let rec 
        pos = function (x) 
                 if (x >= 0)
                 then x
                 else neg(x)
        neg = function (y)
                 if (y <= 0)
                 then pos (1 + y * y)
                 else pos(1 - y)
   in 
     neg(10.0)
~~~

The two functions are _mutually recursive_ since `pos` calls `neg` and vice-versa.
Convince yourself that the program must return `82`.

## 1A (5 points): Extending the Abstract Syntax

Consider the grammar specification we have seen thus far.

$$\begin{array}{rcll}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
 & | & Ident(\mathbf{Identifier}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Mult(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Eq(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Geq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) & \text{if (expr) then expr else expr} \\
 & | & Let( \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) & \text{let identifier = expr in expr} \\
 & | & FunDef( \mathbf{Identifier}, \mathbf{Expr}) & \text{function (identifier-formal-parameter) expr } \\ 
 & | & FunCall(\mathbf{Expr}, \mathbf{Expr}) & \text{function call - expr(expr)} \\
 & | & LetRec(\mathbf{Identifier}, \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr})  \\
\end{array}$$

We wish to add a new rule for two mutually recursive functions

$$ \mathbf{Expr}\ \rightarrow\ LetRec2( \mathbf{Identifier}, \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Identifier}, \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr} ) $$

Such that a mutual call such as

~~~
let rec 
         f1 = function (x1) e1
         f2 = function (x2) e2
     in 
         e3
~~~
is represented in the AST as

~~~
LetRec2(f1, x1, e1, f2, x2, e2, e3)
~~~

Extend the existing AST specification to add support for `LetRec2`.

**Important:** We will use the type `P1Expr` in Scala to represent __Expr__ from the grammar above.

In [9]:
sealed trait Program
sealed trait P1Expr
case class Const(f: Double) extends P1Expr
case class Ident(s: String) extends P1Expr
case class Minus(e1: P1Expr, e2: P1Expr) extends P1Expr
case class Plus(e1: P1Expr, e2: P1Expr) extends P1Expr
case class Mult(e1: P1Expr, e2: P1Expr) extends P1Expr
case class Eq(e1: P1Expr, e2: P1Expr) extends P1Expr
case class Geq(e1: P1Expr, e2: P1Expr) extends P1Expr
case class IfThenElse(e1: P1Expr, e2: P1Expr, e3: P1Expr) extends P1Expr
case class Let(x: String, e1: P1Expr, e2: P1Expr) extends P1Expr
case class FunDef(id: String, e: P1Expr) extends P1Expr
case class FunCall(calledFun: P1Expr, argExpr: P1Expr) extends P1Expr
case class LetRec(funName: String, param: String, funExpr: P1Expr, bodyExpr: P1Expr) extends P1Expr

// YOU CODE HERE
case class LetRec2(funName: String, param: String, funExpr: P1Expr, param2: String, param3: String, funExpr2: P1Expr,bodyExpr: P1Expr) extends P1Expr
case class TopLevel(e: P1Expr) extends Program

defined trait Program
defined trait P1Expr
defined class Const
defined class Ident
defined class Minus
defined class Plus
defined class Mult
defined class Eq
defined class Geq
defined class IfThenElse
defined class Let
defined class FunDef
defined class FunCall
defined class LetRec
defined class LetRec2
defined class TopLevel

In [10]:
//BEGIN TEST
val x = Ident("x")
val y = Ident("y")
val foo = Ident("foo")
val bar = Ident("bar")

val e1 = IfThenElse( Geq(x, Const(0.0)), x, FunCall(bar, Plus(x, Const(1.0)))) // if x >= 0 then x else bar(1 + x)
val e2 = IfThenElse( Geq(Const(1.0), x), Plus(Const(2.0), x), FunCall(foo, Minus(x, Const(2.0)))) // if 1 >= x then 2 + x else foo(x-2)
val e3 = FunCall(bar, Const(10))
val lr2 = LetRec2("foo", "x", e1, "bar", "x", e2, e3)
val p1 = TopLevel(lr2)
passed(3)
//END TEST


*** Tests Passed (3 points) ***


x: Ident = Ident(s = "x")
y: Ident = Ident(s = "y")
foo: Ident = Ident(s = "foo")
bar: Ident = Ident(s = "bar")
e1: IfThenElse = IfThenElse(
  e1 = Geq(e1 = Ident(s = "x"), e2 = Const(f = 0.0)),
  e2 = Ident(s = "x"),
  e3 = FunCall(
    calledFun = Ident(s = "bar"),
    argExpr = Plus(e1 = Ident(s = "x"), e2 = Const(f = 1.0))
  )
)
e2: IfThenElse = IfThenElse(
  e1 = Geq(e1 = Const(f = 1.0), e2 = Ident(s = "x")),
  e2 = Plus(e1 = Const(f = 2.0), e2 = Ident(s = "x")),
  e3 = FunCall(
    calledFun = Ident(s = "foo"),
    argExpr = Minus(e1 = Ident(s = "x"), e2 = Const(f = 2.0))
  )
)
e3: FunCall = FunCall(calledFun = Ident(s = "bar"), argExpr = Const(f = 10.0))
lr2: LetRec2 = LetRec2(
  funName = "foo",
  param = "x",
  funExpr = IfThenElse(
    e1 = Geq(e1 = Ident(s = "x"), e2 = Const(f = 0.0)),
    e2 = Ident(s = "x"),
    e3 = FunCall(
      calledFun = Ident(s = "bar"),
      argExpr = Plus(e1 = Ident(s = "x"), e2 = Const(f = 1.0))
    )
  ),
  param2 = "bar",
  param3 = "x",
  fun

In [11]:
//BEGIN TEST
val x = Ident("x")
val y = Ident("y")
val foo = Ident("foo")
val bar = Ident("bar")
val e11 = IfThenElse(Geq(x, Const(0.0)), FunCall(bar, Minus(Const(1.0), x)),Mult(x, FunCall(foo,  Minus(x, Const(1.0))) ))
val e1 = IfThenElse(Eq(x, Const(0.0)), Const(1.0), e11)
val e2 = IfThenElse(Geq(Const(0.0), y), Mult(Const(-0.5), y) , FunCall(foo, y))
val e3 = FunCall(foo, Const(10.5))
val lr3 = LetRec2("foo", "x", e1, "bar", "y", e2, e3)
passed(2)
//END TEST


*** Tests Passed (2 points) ***


x: Ident = Ident(s = "x")
y: Ident = Ident(s = "y")
foo: Ident = Ident(s = "foo")
bar: Ident = Ident(s = "bar")
e11: IfThenElse = IfThenElse(
  e1 = Geq(e1 = Ident(s = "x"), e2 = Const(f = 0.0)),
  e2 = FunCall(
    calledFun = Ident(s = "bar"),
    argExpr = Minus(e1 = Const(f = 1.0), e2 = Ident(s = "x"))
  ),
  e3 = Mult(
    e1 = Ident(s = "x"),
    e2 = FunCall(
      calledFun = Ident(s = "foo"),
      argExpr = Minus(e1 = Ident(s = "x"), e2 = Const(f = 1.0))
    )
  )
)
e1: IfThenElse = IfThenElse(
  e1 = Eq(e1 = Ident(s = "x"), e2 = Const(f = 0.0)),
  e2 = Const(f = 1.0),
  e3 = IfThenElse(
    e1 = Geq(e1 = Ident(s = "x"), e2 = Const(f = 0.0)),
    e2 = FunCall(
      calledFun = Ident(s = "bar"),
      argExpr = Minus(e1 = Const(f = 1.0), e2 = Ident(s = "x"))
    ),
    e3 = Mult(
      e1 = Ident(s = "x"),
      e2 = FunCall(
        calledFun = Ident(s = "foo"),
        argExpr = Minus(e1 = Ident(s = "x"), e2 = Const(f = 1.0))
      )
    )
  )
)
e2: IfThenElse = IfThenElse(

## 1B (10 points): Build an Environment to Handle Mutual Recursion

In class and notes, we saw how to build an environment that handles recursive calls using the `ExtendRec` construct.

Now we propose an `ExtendMutualRec2` construct such that

$$\begin{array}{c}
\\
\hline
{ \text{eval}(\texttt{LetRec2(f1, x1, e1, f2, x2, e2, e)}, \sigma) = \text{eval}(e, \texttt{ExtendMutualRec2}(\texttt{f1, x1, e1, f2, x2, e2}, \sigma)) } \\
\end{array}(\text{Mutual-Rec-OK})$$


Complete the mathematical specification of $\texttt{sExtendMutualRec2}$. Let $\pi = \texttt{ExtendMutualRec2}(\texttt{f1, x1, e1, f2, x2, e2}, \sigma)$.

$$ \pi(y) = \begin{cases}
\color{red}{1} & \mbox{if}\ y = \texttt{f1} \\
\color{red}{2} & \mbox{if}\ y = \texttt{f2} \\
\color{red}{3} & \text{otherwise}\\
\end{cases}$$

Fill in the appropriate values for  $\color{red}{1}, \color{red}{2}, \color{red}{3}$.

Write your answer in the cell bellow. You can make a numbered list in markdown to represent your answers as follows:
1. First
2. Second
3. And so on...

YOUR ANSWER HERE:

*First: if y = f1: Clouser(x1,e1, pi)

*Second:if y = f2: Clouser(x2,e2, pi) 

*Third: call in lookenv (sigma, y)


## 1C (9 points): Code up the Environment Spec

Implement the environment for `ExtendMutualRec`.

In [12]:
sealed trait Environment 
sealed trait Value

case object EmptyEnv extends Environment
case class Extend(x: String, v: Value, sigma: Environment) extends Environment
case class ExtendRec(f: String, x: String, e: P1Expr, sigma: Environment ) extends Environment
case class ExtendMutualRec2(f1: String, x1: String, e1: P1Expr, f2: String, x2: String, e2: P1Expr, sigma: Environment) extends Environment

/* -- We need to redefine values to accomodate the new representation of environments --*/
case class NumValue(d: Double) extends Value
case class BoolValue(b: Boolean) extends Value
case class Closure(x: String, e: P1Expr, pi: Environment) extends Value
case object ErrorValue extends Value


/*2. Operators on values */

def valueToNumber(v: Value): Double = v match {
    case NumValue(d) => d
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a number")
}

def valueToBoolean(v: Value): Boolean = v match {
    case BoolValue(b) => b
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a boolean")
}

def valueToClosure(v: Value): Closure = v match {
    case Closure(x, e, pi) => Closure(x, e, pi)
    case _ =>  throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a closure")
}


/*-- Operations on environments --*/

def lookupEnv(pi: Environment, x: String): Value = pi match {
    case EmptyEnv => throw new IllegalArgumentException(s"Error could not find string $x in environment")
    case Extend(y, v, _) if y == x => v
    case Extend(_, _, sigma) => lookupEnv(sigma, x)
    case ExtendRec(f, y, e, sigma) => {
        if (x == f) 
            Closure(y, e, pi)
        else
            lookupEnv(sigma , x)
    }
    case ExtendMutualRec2(f1, x1, e1, f2, x2, e2, sigma ) => 
    {
        if(x == f1)
            Closure(x1, e1,pi)
        else if(x == f2)
            Closure(x2, e2,pi)
        else
            lookupEnv(sigma, x)
    }
}

defined trait Environment
defined trait Value
defined object EmptyEnv
defined class Extend
defined class ExtendRec
defined class ExtendMutualRec2
defined class NumValue
defined class BoolValue
defined class Closure
defined object ErrorValue
defined function valueToNumber
defined function valueToBoolean
defined function valueToClosure
defined function lookupEnv

In [13]:
// BEGIN TEST
val env: Environment = ExtendMutualRec2("f1", "x1", Const(0.0), "f2", "x2", Const(0.0), EmptyEnv)
passed(4)
// END TEST


*** Tests Passed (4 points) ***


env: Environment = ExtendMutualRec2(
  f1 = "f1",
  x1 = "x1",
  e1 = Const(f = 0.0),
  f2 = "f2",
  x2 = "x2",
  e2 = Const(f = 0.0),
  sigma = EmptyEnv
)

In [14]:
// BEGIN TEST
val env: Environment = ExtendMutualRec2("f1", "x1", Const(0.0), "f2", "x2", Const(0.0), EmptyEnv)
// Ensure looking up either function gets us the right value, no matter how many times we recurse.
val f1 @ Closure(_, _, pi1) = lookupEnv(env, "f1")
val f2 @ Closure(_, _, pi2) = lookupEnv(env, "f2")
lookupEnv(pi1, "f1") == f1
lookupEnv(pi1, "f2") == f2
lookupEnv(pi2, "f2") == f1
lookupEnv(pi2, "f2") == f2
passed(5)
// END TEST


*** Tests Passed (5 points) ***


env: Environment = ExtendMutualRec2(
  f1 = "f1",
  x1 = "x1",
  e1 = Const(f = 0.0),
  f2 = "f2",
  x2 = "x2",
  e2 = Const(f = 0.0),
  sigma = EmptyEnv
)
f1: Closure = Closure(
  x = "x1",
  e = Const(f = 0.0),
  pi = ExtendMutualRec2(
    f1 = "f1",
    x1 = "x1",
    e1 = Const(f = 0.0),
    f2 = "f2",
    x2 = "x2",
    e2 = Const(f = 0.0),
    sigma = EmptyEnv
  )
)
pi1: Environment = ExtendMutualRec2(
  f1 = "f1",
  x1 = "x1",
  e1 = Const(f = 0.0),
  f2 = "f2",
  x2 = "x2",
  e2 = Const(f = 0.0),
  sigma = EmptyEnv
)
f2: Closure = Closure(
  x = "x2",
  e = Const(f = 0.0),
  pi = ExtendMutualRec2(
    f1 = "f1",
    x1 = "x1",
    e1 = Const(f = 0.0),
    f2 = "f2",
    x2 = "x2",
    e2 = Const(f = 0.0),
    sigma = EmptyEnv
  )
)
pi2: Environment = ExtendMutualRec2(
  f1 = "f1",
  x1 = "x1",
  e1 = Const(f = 0.0),
  f2 = "f2",
  x2 = "x2",
  e2 = Const(f = 0.0),
  sigma = EmptyEnv
)
res13_3: Boolean = true
res13_4: Boolean = true
res13_5: Boolean = false
res13_6: Boolean = tr

## 1D (6 points): Interpreter

Complete the interpreter for this new node.

In [15]:

/*
    case object EmptyEnv extends Environment
    case class Extend(x: String, v: Value, sigma: Environment) extends Environment
    case class ExtendRec(f: String, x: String, e: P1Expr, sigma: Environment ) extends Environment
    case class ExtendMutualRec2(f1: String, x1: String, e1: P1Expr, f2: String, x2: String, e2: P1Expr, sigma: Environment) extends Environment

    /* -- We need to redefine values to accomodate the new representation of environments --*/
    case class NumValue(d: Double) extends Value
    case class BoolValue(b: Boolean) extends Value
    case class Closure(x: String, e: P1Expr, pi: Environment) extends Value
    case object ErrorValue extends Value
*/
def evalExpr(e: P1Expr, env: Environment): Value =  {
    
    /* Method to deal with binary arithmetic operations */
    def applyArith2 (e1: P1Expr, e2: P1Expr) (fun: (Double , Double) => Double) = {
        val v1 = valueToNumber(evalExpr(e1, env))
        val v2 = valueToNumber(evalExpr(e2, env))
        val v3 = fun(v1, v2)
        NumValue(v3)
    }  /* -- We have deliberately curried the method --*/
    
    /* Helper method to deal with unary arithmetic */
    def applyArith1(e: P1Expr) (fun: Double => Double) = {
        val v = valueToNumber(evalExpr(e, env))
        val v1 = fun(v)
        NumValue(v1)
    }
    
    /* Helper method to deal with comparison operators */
    def applyComp(e1: P1Expr, e2: P1Expr) (fun: (Double, Double) => Boolean) = {
        val v1 = valueToNumber(evalExpr(e1, env))
        val v2 = valueToNumber(evalExpr(e2, env))
        val v3 = fun(v1, v2)
        BoolValue(v3)
    }
    
   
    e match {
        case Const(f) => NumValue(f) // Same as before
        
        case Ident(x) => lookupEnv(env, x) // Changed to accomodate the new environment definitions.
    
        /* Ditto as before */
        case Plus(e1, e2) => applyArith2 (e1, e2) ( _ + _ )
        /* Ditto as before */
        case Minus(e1, e2) => applyArith2(e1, e2) ( _ - _ )
        /* Ditto as before */
        case Mult(e1, e2) =>  applyArith2(e1, e2) (_ * _)
        /* Ditto as before */
        case Geq(e1, e2) => applyComp(e1, e2)(_ >= _)
        /* Ditto as before */
        case Eq(e1, e2) => applyComp(e1, e2)(_ == _)
        /* Ditto as before */
        case IfThenElse(e1, e2, e3) => {
            val v = evalExpr(e1, env)
            v match {
                case BoolValue(true) => evalExpr(e2, env)
                case BoolValue(false) => evalExpr(e3, env)
                case _ => throw new IllegalArgumentException(s"If-then-else condition P1Expr: ${e1} is non-boolean -- evaluates to ${v}")
            }
        }
        /* Ditto as before */
        case Let(x, e1, e2) => {
            val v1 = evalExpr(e1, env)  // eval e1
            val env2 = Extend(x, v1, env) // create a new extended env
            evalExpr(e2, env2) // eval e2 under that.
        }
        /* Ditto as before */
        case FunDef(x, e) => {
            Closure(x, e, env) // Return a closure with the current enviroment.
        }
        /* Ditto as before */
        case FunCall(e1, e2) => {
            val v1 = evalExpr(e1, env)
            val v2 = evalExpr(e2, env)
            v1 match {
                case Closure(x, closure_ex, closed_env) => {
                    // First extend closed_env by binding x to v2
                    val new_env = Extend(x, v2, closed_env)
                    // Evaluate the body of the closure under the extended environment.
                    evalExpr(closure_ex, new_env)
                }
                case _ => throw new IllegalArgumentException(s"Function call error: P1Expression $e1 does not evaluate to a closure")
            }
        }
    
        case LetRec(rfun, x, fExpr, bExpr) => {
            val env2 = ExtendRec(rfun, x, fExpr, env)
            evalExpr(bExpr, env2)
        }
        
        case LetRec2(f1, x1, e1, f2, x2, e2, e) => { // e1 is the body of of f1 and e2 is the body for f2
            // YOUR CODE HERE
            val new_env = ExtendMutualRec2(f1, x1, e1, f2, x2, e2, env) //new_Env is a enironment
            evalExpr(e, new_env) //
        }
    }
}

def evalProgram(p: Program) = {
    p match { 
        case TopLevel(e) => evalExpr(e, EmptyEnv)
    }
}


defined function evalExpr
defined function evalProgram

In [16]:
//BEGIN TEST
val x = Ident("x")
val y = Ident("y")
val foo = Ident("foo")
val bar = Ident("bar")

val e1 = IfThenElse( Geq(x, Const(0.0)), x, FunCall(bar, Plus(x, Const(1.0)))) // if x >= 0 then x else bar(1 + x)
val e2 = IfThenElse( Geq(Const(1.0), x), Plus(Const(2.0), x), FunCall(foo, Minus(x, Const(2.0)))) // if 1 >= x then 2 + x else foo(x-2)
val e3 = FunCall(bar, Const(10))

val lr2 = LetRec2("foo", "x", e1, "bar", "x", e2, e3)
val p1 = TopLevel(lr2)
assert(evalProgram(p1) == NumValue(8.0), "Test 1 of Set 1 failed")

val e4 = FunCall(foo, Const(12.0))
val p2 = TopLevel(LetRec2("foo", "x", e1, "bar", "x", e2, e4))
assert(evalProgram(p2) == NumValue(12.0), "Test 2 of Set 1 failed")

val e5 = FunCall(foo, Const(-12.0))
val p3 = TopLevel(LetRec2("foo", "x", e1, "bar", "x", e2, e5))
assert(evalProgram(p3) == NumValue(-9.0), "Test 3 of Set 1 failed")

val e6 = FunCall(bar, Const(-12.0))
val p4 = TopLevel(LetRec2("foo", "x", e1, "bar", "x", e2, e6))
assert(evalProgram(p4) == NumValue(-10.0), "Test 4 of Set 1 failed")

val e7 = FunCall(bar, Const(1.9))
val p5 = TopLevel(LetRec2("foo", "x", e1, "bar", "x", e2, e7))
assert(evalProgram(p5) == NumValue(2.9), "Test 5 of Set 1 failed")

passed(3)
//END TEST


*** Tests Passed (3 points) ***


x: Ident = Ident(s = "x")
y: Ident = Ident(s = "y")
foo: Ident = Ident(s = "foo")
bar: Ident = Ident(s = "bar")
e1: IfThenElse = IfThenElse(
  e1 = Geq(e1 = Ident(s = "x"), e2 = Const(f = 0.0)),
  e2 = Ident(s = "x"),
  e3 = FunCall(
    calledFun = Ident(s = "bar"),
    argExpr = Plus(e1 = Ident(s = "x"), e2 = Const(f = 1.0))
  )
)
e2: IfThenElse = IfThenElse(
  e1 = Geq(e1 = Const(f = 1.0), e2 = Ident(s = "x")),
  e2 = Plus(e1 = Const(f = 2.0), e2 = Ident(s = "x")),
  e3 = FunCall(
    calledFun = Ident(s = "foo"),
    argExpr = Minus(e1 = Ident(s = "x"), e2 = Const(f = 2.0))
  )
)
e3: FunCall = FunCall(calledFun = Ident(s = "bar"), argExpr = Const(f = 10.0))
lr2: LetRec2 = LetRec2(
  funName = "foo",
  param = "x",
  funExpr = IfThenElse(
    e1 = Geq(e1 = Ident(s = "x"), e2 = Const(f = 0.0)),
    e2 = Ident(s = "x"),
    e3 = FunCall(
      calledFun = Ident(s = "bar"),
      argExpr = Plus(e1 = Ident(s = "x"), e2 = Const(f = 1.0))
    )
  ),
  param2 = "bar",
  param3 = "x",
  fun

In [17]:
//BEGIN TEST 
/*
let rec 
        pos = function (x) 
                 if (x >= 0)
                 then
                    if (x <= 0.5)
                     then x
                     else 0.5 + pos(x-1)
                 else neg(x)
        neg = function (y)
                 if (y <= 0)
                 then pos (1 + y * y)
                 else pos(1 - y)
   in 
     ...
*/

val x = Ident("x")
val y = Ident("y")
val pos = Ident("pos")
val neg = Ident("neg")

val e11 = IfThenElse(Geq(Const(0.5), x), x, Plus(Const(0.5), FunCall(pos, Minus(x, Const(1.0)))))
val e1 = IfThenElse(Geq(x, Const(0.0)), e11, FunCall(neg, x))
val e2 = IfThenElse(Geq(Const(0.0), y), FunCall(pos, Plus(Const(1.0), Mult(y,y))), FunCall(pos, Minus(Const(1.0), y)))

val t1 = FunCall(neg, Const(10.0))
val lr1 = LetRec2("pos", "x", e1, "neg", "y", e2, t1)
val p1 = TopLevel(lr1)
assert(evalProgram(p1) == NumValue(41.0), "Test 1 of set 2 failed")

val t2 = FunCall(pos, Const(-8.0))
val lr2 = LetRec2("pos", "x", e1, "neg", "y", e2, t2)
val p2 = TopLevel(lr2)
assert(evalProgram(p2) == NumValue(32.5), "Test 2 of set 2 failed")


val t3 = FunCall(pos, Const(-0.5))
val lr3 = LetRec2("pos", "x", e1, "neg", "y", e2, t3)
val p3 = TopLevel(lr3)
assert(evalProgram(p3) == NumValue(0.75), "Test 3 of set 2 failed")

passed(3)
//END TEST


*** Tests Passed (3 points) ***


x: Ident = Ident(s = "x")
y: Ident = Ident(s = "y")
pos: Ident = Ident(s = "pos")
neg: Ident = Ident(s = "neg")
e11: IfThenElse = IfThenElse(
  e1 = Geq(e1 = Const(f = 0.5), e2 = Ident(s = "x")),
  e2 = Ident(s = "x"),
  e3 = Plus(
    e1 = Const(f = 0.5),
    e2 = FunCall(
      calledFun = Ident(s = "pos"),
      argExpr = Minus(e1 = Ident(s = "x"), e2 = Const(f = 1.0))
    )
  )
)
e1: IfThenElse = IfThenElse(
  e1 = Geq(e1 = Ident(s = "x"), e2 = Const(f = 0.0)),
  e2 = IfThenElse(
    e1 = Geq(e1 = Const(f = 0.5), e2 = Ident(s = "x")),
    e2 = Ident(s = "x"),
    e3 = Plus(
      e1 = Const(f = 0.5),
      e2 = FunCall(
        calledFun = Ident(s = "pos"),
        argExpr = Minus(e1 = Ident(s = "x"), e2 = Const(f = 1.0))
      )
    )
  ),
  e3 = FunCall(calledFun = Ident(s = "neg"), argExpr = Ident(s = "x"))
)
e2: IfThenElse = IfThenElse(
  e1 = Geq(e1 = Const(f = 0.0), e2 = Ident(s = "y")),
  e2 = FunCall(
    calledFun = Ident(s = "pos"),
    argExpr = Plus(
      e1 = Const(f 

## Problem 2 (15 Points): Auto-Curry

In class, we have explored just single argument functions in lettuce. In this problem, we will 
explore, multiple argument functions.

Consider: 

~~~
let foo1 = function (x, y)
            x - 2 * y
          in 
       foo1(10, 15)
~~~

this code should return -20

We will allow zero arguments as well.

~~~
let x = 5 in 
let bar1 = function()
           x 
           in 
    bar1()
~~~

This code should return 5.

## Extending the Abstract Syntax

Consider the grammar specification we have seen thus far.

$$\begin{array}{rcll}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
 & | & Ident(\mathbf{Identifier}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & FunDef( \mathbf{Identifier}, \mathbf{Expr}) & \text{Just one parameter } \\ 
 & | & FunCall(\mathbf{Expr}, \mathbf{Expr}) & \text{function call - for just one parameter} \\
 & | & FunDefMulti( \mathbf{Identifier}*, \mathbf{Expr}) & \text{Note multiple parameters possible}\\
 & | &  FunCallMulti(\mathbf{Expr}, \mathbf{Expr}^*) & \text{function call with multiple expressions - expr(expr1, ... , exprn)} \\
 & | & Let(\mathbf{Identifier},\mathbf{Expr}, \mathbf{Expr})  \\
\end{array}$$

We now provide Scala definition for the grammar above. We use lists to implement the Kleene Star.
Please ensure you run this cell

In [18]:
sealed trait Program
sealed trait Expr
case class Const(f: Double) extends Expr
case class Ident(s: String) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Let(x: String, e1: Expr, e2: Expr) extends Expr
case class FunDef(id: String, e: Expr) extends Expr
case class FunCall(calledFun: Expr, arg: Expr) extends Expr
case class FunDefMulti(idList: List[String], e: Expr) extends Expr
case class FunCallMulti(calledFun: Expr, argList: List[Expr]) extends Expr
case class TopLevel(e: Expr) extends Program

defined trait Program
defined trait Expr
defined class Const
defined class Ident
defined class Plus
defined class Let
defined class FunDef
defined class FunCall
defined class FunDefMulti
defined class FunCallMulti
defined class TopLevel

The goal is to convert functions with multiple arguments into curried functions and likewise transform function calls.

### Example -1 

~~~
 let f = function (x, y) 
            x - y 
        in 
     f (10, 20)
~~~

Will transform into 

~~~
 let f = function (x) 
            function (y)
               x - y 
       in 
    f(10)(20)
~~~

### Example-2

~~~
 let f = function () 10 in 
    f()
~~~

will transform into 

~~~
  let f = function (__dummy) 10 in 
      f (-1)
~~~

Note that the `__dummy` parameter you introduce should not be used in the code. However, you may safely assume some variable name such as `__dummy` is used nowhere else. Likewise for function calls with no arguments, the exact value we pass into the dummy parameter does not matter. 


### Semantics of AutoCurry


$$\newcommand\semrule[3]{\begin{array}{c} #1 \\ \hline #2 \\ \end{array}(\text{#3})} $$
$$\newcommand\autoCurry{\textsf{AutoCurry}}$$
Here are the semantic rules for the $\autoCurry$ function which takes in an expression and returns
an expression.

$$\semrule{}{\autoCurry( \texttt{Const(f)} ) = \texttt{Const(f)} } {autocurry-const} $$
$$\semrule{}{\autoCurry( \texttt{Ident(x)} ) = \texttt{Ident(x)} } {autocurry-ident} $$
$$\semrule{\autoCurry(\texttt{e1}) = \texttt{f1}, \autoCurry(\texttt{e2}) = \texttt{f2}}{\autoCurry( \texttt{Plus(e1, e2)} ) = \texttt{Plus(f1, f2)} } {autocurry-plus} $$
$$\semrule{\autoCurry(\texttt{e1}) = \texttt{f1}, \autoCurry(\texttt{e2}) = \texttt{f2}}{\autoCurry( \texttt{Let(x, e1, e2)} ) = \texttt{Let(x, f1, f2)} } {autocurry-let} $$
$$\semrule{\autoCurry(\texttt{e}) = \texttt{f}}{\autoCurry( \texttt{FunDef(x, e)} ) = \texttt{FunDef(x, f)} } {autocurry-fundef} $$
$$\semrule{\autoCurry(\texttt{e1}) = \texttt{f1}, \autoCurry(\texttt{e2}) = \texttt{f2}}{\autoCurry( \texttt{FunCall(e1, e2)} ) = \texttt{FunCall(f1, f2)} } {autocurry-funcall} $$

The important rules are for `FunDefMulti` and `FunCallMulti`: 

$$\semrule{\autoCurry(\texttt{e}) = \texttt{f}, n \geq 1}{\autoCurry( \texttt{FunDefMulti([x1, x2,..,xn], e)} ) = \texttt{FunDef(x1, FunDef( x2, ..., FunDef(xn, f))))} } {autocurry-fundef-one-or-more-params} $$

Let `_dummy_` be a string that does not appear as an identifier anywhere in the expression `e`. 
$$\semrule{\autoCurry(\texttt{e}) = \texttt{f}}{\autoCurry( \texttt{FunDefMulti([], e)} ) = \texttt{FunDef}(\text{_dummy_}, \texttt{f})}  {autocurry-fundef-zero-params} $$



$$\semrule{\autoCurry(\texttt{eCall}) = \texttt{fCall},\ n \geq 1,\ \autoCurry(\texttt{e1}) = \texttt{f1}, \ldots, \autoCurry(\texttt{en}) = \texttt{fn} }{\autoCurry( \texttt{FunCallMulti(eCall, [e1, ..., en])} ) = \texttt{FunCall}( \cdots \texttt{FunCall}(\texttt{FunCall}(\texttt{fCall}, \texttt{f1}), \texttt{f2}) \cdots, , \texttt{fn}) } {autocurry-funcall-one-or-more-params} $$

$$\semrule{\autoCurry(\texttt{eCall}) = \texttt{fCall} }{\autoCurry( \texttt{FunCallMulti(eCall, [])} ) = \texttt{FunCall}(\texttt{fCall}, \texttt{Const}(-1)) } {autocurry-funcall-no-params} $$

Once an expression has been completely "autoCurried", there are no more instances of `FunDefMulti` or `FunCallMulti` left.

Implement the function `autoCurry(e: Expr): Expr` below.

In [19]:
def autoCurryProgram(p: Program ): Program = p match {
    case TopLevel(e) => TopLevel(autoCurry(e))
}
// YOUR CODE HERE
def autoCurry(e: Expr): Expr = e match{
    case Const(f) => Const(f)
    case Ident(f) => Ident(f)
    case Plus(e1, e2) => Plus(autoCurry(e1),autoCurry(e2))
    case Let(x, e1, e2) => Let(x, autoCurry(e1), autoCurry(e2))
    case FunDef(x, e1) => FunDef(x, autoCurry(e1))
    case FunCall(e1, e2) => FunCall(autoCurry(e1), autoCurry(e2))
    case FunDefMulti(expr:List[String], e1:Expr)=> 
    {
        val f = autoCurry(e1) // Curring function
        val new_expr = expr.reverse
        new_expr.foldLeft(f) { 
            (acc, id) => FunDef(id, acc)
        }
        
    }
     case FunCallMulti(e1, expr)=>
    {
        val f = autoCurry(e1)
        expr.foldLeft(f)
        {
            (v1, v2) => FunCall(v1, v2)
        }
    }
    
}

defined function autoCurryProgram
defined function autoCurry

### Important

Please run the cell below to define `evalForAutoCurry`  and `testSanity` before we proceed.

In [20]:
case class RuntimeError(s: String) extends Exception {
    override def toString(): String = f"Runtime error: $s"
}

sealed trait Value
case class Num(f: Double) extends Value
case class Closure(x: String, e:Expr, env: Map[String, Value]) extends Value 

def binop(v1: Value)(v2: Value)(op: (Double, Double) => Double) = (v1, v2) match {
    case (Num(f1), Num(f2)) => Num(op(f1, f2))
    case _ =>  throw new RuntimeError("Runtime type mismatch")
}
def evalForAutoCurry(e: Expr, env: Map[String, Value]): Value = e match {
    case Const(f) => Num(f)
    case Ident(x) => if (env contains x) { env(x)} else { throw new RuntimeError("Unknown identifer")}
    case Plus(e1, e2) => binop(evalForAutoCurry(e1, env))(evalForAutoCurry(e2, env))(_ + _)
    case Let(x, e1, e2) => {
        val v1 = evalForAutoCurry(e1, env)
        evalForAutoCurry(e2, env +(x -> v1))
    }
    case FunDef(x, e) => Closure(x, e, env)
    case FunCall(e1, e2) => {
        val v1 = evalForAutoCurry(e1, env)
        v1 match {
            case Closure(p, eBody, envClosure) => {
                val v2 = evalForAutoCurry(e2, env)
                evalForAutoCurry(eBody, envClosure+( p -> v2))
            }
            case _ => throw new RuntimeError("Runtime function call type mismatch")
        }
    }
    case FunDefMulti(_, _) => throw new RuntimeError("You were supposed to get rid of multiple function calls")
    case FunCallMulti(_, _) => throw new RuntimeError("You were supposed to get rid of multiple function calls")
    
}


def testResultHasNoMultiFun(e: Expr): Boolean = e match {
    case Const(_) => true
    case Ident(_) => true
    case Plus(e1, e2) => testResultHasNoMultiFun(e1) && testResultHasNoMultiFun(e2)
    case Let(x, e1, e2) => testResultHasNoMultiFun(e1) && testResultHasNoMultiFun(e2)
    case FunDef(x, e1) => testResultHasNoMultiFun(e1)
    case FunCall(e1, e2) => testResultHasNoMultiFun(e1) && testResultHasNoMultiFun(e2)
    case FunDefMulti(_, _) => throw new RuntimeError("You were supposed to get rid of multiple function calls")
    case FunCallMulti(_, _) => throw new RuntimeError("You were supposed to get rid of multiple function calls")
    
}

defined class RuntimeError
defined trait Value
defined class Num
defined class Closure
defined function binop
defined function evalForAutoCurry
defined function testResultHasNoMultiFun

In [21]:
val x1: Expr = Ident("x1")
val x2 : Expr= Ident("x2")
val x3 : Expr= Ident("x3")

// function (x1, x2, x3) (x1 + x2 + x3)

val e1: Expr = FunDefMulti(List("x1", "x2", "x3"), Plus(x1, Plus(x2, x3)) )

val f1: Expr = autoCurry(e1)

println("Checking that result from your function has no multiple arg functions left.")
testResultHasNoMultiFun(f1)
println("Success!!")
println("Checking result is correct:")
// check that result is function (x1) function (x2) function (x3) x1 + x2 + x3
assert(f1 == FunDef("x1", FunDef("x2", FunDef("x3",Plus(x1, Plus(x2, x3))  ))))

passed(3)

Checking that result from your function has no multiple arg functions left.
Success!!
Checking result is correct:

*** Tests Passed (3 points) ***


x1: Expr = Ident(s = "x1")
x2: Expr = Ident(s = "x2")
x3: Expr = Ident(s = "x3")
e1: Expr = FunDefMulti(
  idList = List("x1", "x2", "x3"),
  e = Plus(
    e1 = Ident(s = "x1"),
    e2 = Plus(e1 = Ident(s = "x2"), e2 = Ident(s = "x3"))
  )
)
f1: Expr = FunDef(
  id = "x1",
  e = FunDef(
    id = "x2",
    e = FunDef(
      id = "x3",
      e = Plus(
        e1 = Ident(s = "x1"),
        e2 = Plus(e1 = Ident(s = "x2"), e2 = Ident(s = "x3"))
      )
    )
  )
)
res20_6: Boolean = true

In [22]:
val x1: Expr = Ident("x1")
val x2: Expr = Ident("x2")
val x3: Expr = Ident("x3")
// ( (x1, x2, x3) => (x1+ x2+ x3 + x3)) (1, 2, 3)
val e1: Expr = FunDefMulti(List("x1", "x2", "x3"), Plus(x1, Plus(x2, Plus(x3, x3)) ))
val e2: Expr = FunCallMulti(e1, List(Const(1), Const(2), Const(3)))

val f2: Expr = autoCurry(e2)

println("Checking that result from your function has no multiple arg functions left.")
testResultHasNoMultiFun(f2)
println("Success!!")
println("Checking result is correct:")
assert(evalForAutoCurry(f2, Map.empty)== Num(9.0))
passed(3)

Checking that result from your function has no multiple arg functions left.
Success!!
Checking result is correct:

*** Tests Passed (3 points) ***


x1: Expr = Ident(s = "x1")
x2: Expr = Ident(s = "x2")
x3: Expr = Ident(s = "x3")
e1: Expr = FunDefMulti(
  idList = List("x1", "x2", "x3"),
  e = Plus(
    e1 = Ident(s = "x1"),
    e2 = Plus(
      e1 = Ident(s = "x2"),
      e2 = Plus(e1 = Ident(s = "x3"), e2 = Ident(s = "x3"))
    )
  )
)
e2: Expr = FunCallMulti(
  calledFun = FunDefMulti(
    idList = List("x1", "x2", "x3"),
    e = Plus(
      e1 = Ident(s = "x1"),
      e2 = Plus(
        e1 = Ident(s = "x2"),
        e2 = Plus(e1 = Ident(s = "x3"), e2 = Ident(s = "x3"))
      )
    )
  ),
  argList = List(Const(f = 1.0), Const(f = 2.0), Const(f = 3.0))
)
f2: Expr = FunCall(
  calledFun = FunCall(
    calledFun = FunCall(
      calledFun = FunDef(
        id = "x1",
        e = FunDef(
          id = "x2",
          e = FunDef(
            id = "x3",
            e = Plus(
              e1 = Ident(s = "x1"),
              e2 = Plus(
                e1 = Ident(s = "x2"),
                e2 = Plus(e1 = Ident(s = "x3"), e2 = Ident(s 

In [23]:
val e1: Expr = FunDefMulti(Nil, Const(15))
val e2 : Expr= FunCallMulti(Ident("foo"), Nil)
val e: Expr = Let("foo", e1, e2)

// let foo = function () 15 in foo()
val f: Expr = autoCurry(e)


println("Checking that result from your function has no multiple arg functions left.")
testResultHasNoMultiFun(f)
println("Success!!")
println("Checking result is correct:")
assert(evalForAutoCurry(f, Map.empty) == Num(15.0))
passed(3)

Checking that result from your function has no multiple arg functions left.
Success!!
Checking result is correct:

*** Tests Passed (3 points) ***


e1: Expr = FunDefMulti(idList = List(), e = Const(f = 15.0))
e2: Expr = FunCallMulti(calledFun = Ident(s = "foo"), argList = List())
e: Expr = Let(
  x = "foo",
  e1 = FunDefMulti(idList = List(), e = Const(f = 15.0)),
  e2 = FunCallMulti(calledFun = Ident(s = "foo"), argList = List())
)
f: Expr = Let(x = "foo", e1 = Const(f = 15.0), e2 = Ident(s = "foo"))
res22_5: Boolean = true

In [24]:
val x1: Expr = Ident("x1")
val x2 : Expr= Ident("x2")
val x3: Expr = Ident("x3")

/*
  let foo = function (x) {
           (function (x1, x2, x3) x1 + x2 + x3 + x3) (1, 2, 3) + x
  } in 
    foo(10)
*/

val e1: Expr = FunDefMulti(List("x1", "x2", "x3"), Plus(x1, Plus(x2, Plus(x3, x3)) ))
val e2: Expr = FunCallMulti(e1, List(Const(1), Const(2), Const(3)))

val e3 : Expr= FunDef("x", Plus(e2, Ident("x")))

val e4 : Expr= Let("foo", e3 , FunCall(Ident("foo"), Const(10)))

val f4 : Expr= autoCurry(e4)


println("Checking that result from your function has no multiple arg functions left.")
testResultHasNoMultiFun(f4)
println("Success!!")
println("Checking result is correct:")
assert( evalForAutoCurry(f4, Map.empty) == Num(19.0))

passed(3)

Checking that result from your function has no multiple arg functions left.
Success!!
Checking result is correct:

*** Tests Passed (3 points) ***


x1: Expr = Ident(s = "x1")
x2: Expr = Ident(s = "x2")
x3: Expr = Ident(s = "x3")
e1: Expr = FunDefMulti(
  idList = List("x1", "x2", "x3"),
  e = Plus(
    e1 = Ident(s = "x1"),
    e2 = Plus(
      e1 = Ident(s = "x2"),
      e2 = Plus(e1 = Ident(s = "x3"), e2 = Ident(s = "x3"))
    )
  )
)
e2: Expr = FunCallMulti(
  calledFun = FunDefMulti(
    idList = List("x1", "x2", "x3"),
    e = Plus(
      e1 = Ident(s = "x1"),
      e2 = Plus(
        e1 = Ident(s = "x2"),
        e2 = Plus(e1 = Ident(s = "x3"), e2 = Ident(s = "x3"))
      )
    )
  ),
  argList = List(Const(f = 1.0), Const(f = 2.0), Const(f = 3.0))
)
e3: Expr = FunDef(
  id = "x",
  e = Plus(
    e1 = FunCallMulti(
      calledFun = FunDefMulti(
        idList = List("x1", "x2", "x3"),
        e = Plus(
          e1 = Ident(s = "x1"),
          e2 = Plus(
            e1 = Ident(s = "x2"),
            e2 = Plus(e1 = Ident(s = "x3"), e2 = Ident(s = "x3"))
          )
        )
      ),
      argList = List(Const(f = 1.0), Cons

In [25]:
val x1 : Expr= Ident("x1")
val x2 : Expr= Ident("x2")
val x3 : Expr= Ident("x3")
val x4 : Expr= Ident("x4")
val x5 : Expr= Ident("x5")


/*
  let foo = function (x1, x2) {
           (function (x3, x4, x5) x1 + x2 + x2 + x3 + x4 + x5 + x5 )
  } in 
    foo(10,20)(30, 40, 50)
*/

val e1: Expr = FunDefMulti(List("x1", "x2"), FunDefMulti(List("x3", "x4", "x5"), 
                                                   Plus(x1, 
                                                        Plus(x2, Plus(x2, 
                                                            Plus(x3, 
                                                                 Plus(x4, Plus(x5, x5))
                                                                )))
                                                       ) ))

val e2 : Expr= FunCallMulti(Ident("foo"), List(Const(10), Const(20)))

val e3: Expr = FunCallMulti(e2, List(Const(30), Const(40), Const(50)))

val e4: Expr = Let("foo", e1 , e3)

val f4: Expr = autoCurry(e4)

println("Checking that result from your function has no multiple arg functions left.")
testResultHasNoMultiFun(f4)
println("Success!!")
println("Checking result is correct:")

assert( evalForAutoCurry(f4, Map.empty) == Num(220.0))

passed(3)

Checking that result from your function has no multiple arg functions left.
Success!!
Checking result is correct:

*** Tests Passed (3 points) ***


x1: Expr = Ident(s = "x1")
x2: Expr = Ident(s = "x2")
x3: Expr = Ident(s = "x3")
x4: Expr = Ident(s = "x4")
x5: Expr = Ident(s = "x5")
e1: Expr = FunDefMulti(
  idList = List("x1", "x2"),
  e = FunDefMulti(
    idList = List("x3", "x4", "x5"),
    e = Plus(
      e1 = Ident(s = "x1"),
      e2 = Plus(
        e1 = Ident(s = "x2"),
        e2 = Plus(
          e1 = Ident(s = "x2"),
          e2 = Plus(
            e1 = Ident(s = "x3"),
            e2 = Plus(
              e1 = Ident(s = "x4"),
              e2 = Plus(e1 = Ident(s = "x5"), e2 = Ident(s = "x5"))
            )
          )
        )
      )
    )
  )
)
e2: Expr = FunCallMulti(
  calledFun = Ident(s = "foo"),
  argList = List(Const(f = 10.0), Const(f = 20.0))
)
e3: Expr = FunCallMulti(
  calledFun = FunCallMulti(
    calledFun = Ident(s = "foo"),
    argList = List(Const(f = 10.0), Const(f = 20.0))
  ),
  argList = List(Const(f = 30.0), Const(f = 40.0), Const(f = 50.0))
)
e4: Expr = Let(
  x = "foo",
  e1 = FunDefMulti(
    

### That's all folks